# Preprocessing-Slight

In [92]:
import pandas as pd
import numpy as np
import ast

lab = pd.read_csv("../csv/validated_lab/validated_lab2.csv")
lab = lab.reset_index(drop=True)
lab

,Unnamed: 0,contours,area_hist_values,int_hist_values,plant
0,0,"[[[[144, 110]], [[143, 111]], [[142, 111]], [[...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[20.0, 22.0, 51.0, 41.0, 46.0, 35.0, 29.0, 25....",acer_campestre
1,1,"[[[[158, 107]], [[157, 108]], [[156, 108]], [[...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[21.0, 31.0, 59.0, 52.0, 40.0, 41.0, 24.0, 21....",acer_campestre
2,2,"[[[[142, 86]], [[141, 87]], [[140, 87]], [[139...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0, 23.0, 54.0, 34.0, 49.0, 33.0, 30.0, 24.0...",acer_campestre
3,3,"[[[[162, 85]], [[161, 86]], [[160, 86]], [[159...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[22.0, 34.0, 54.0, 48.0, 46.0, 40.0, 24.0, 8.0...",acer_campestre
4,4,"[[[[185, 100]], [[184, 101]], [[183, 101]], [[...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 15.0, 27.0, 45.0, 40.0, 40.0, 34.0, 18.0...",acer_campestre
...,...,...,...,...,...
13516,13516,"[[[[54, 74]], [[53, 75]], [[52, 75]], [[51, 76...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.0,...","[39.0, 64.0, 69.0, 89.0, 57.0, 52.0, 17.0, 2.0...",quercus_robur
13517,13517,"[[[[115, 78]], [[114, 79]], [[113, 79]], [[112...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0,...","[28.0, 68.0, 48.0, 69.0, 68.0, 54.0, 14.0, 3.0...",quercus_robur
13518,13518,"[[[[94, 101]], [[93, 102]], [[92, 103]], [[91,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0,...","[22.0, 53.0, 69.0, 75.0, 50.0, 42.0, 17.0, 11....",quercus_robur
13519,13519,"[[[[116, 79]], [[115, 80]], [[114, 81]], [[113...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 23.0,...","[34.0, 84.0, 62.0, 59.0, 58.0, 51.0, 15.0, 5.0...",quercus_robur


In [110]:
lab_int = [ast.literal_eval(d) for d in lab['int_hist_values']]
lab = pd.DataFrame({'int_hist_values' : lab_int, 'plant' : lab['plant']})

In [111]:
lab

,int_hist_values,plant
0,"[20.0, 22.0, 51.0, 41.0, 46.0, 35.0, 29.0, 25....",acer_campestre
1,"[21.0, 31.0, 59.0, 52.0, 40.0, 41.0, 24.0, 21....",acer_campestre
2,"[7.0, 23.0, 54.0, 34.0, 49.0, 33.0, 30.0, 24.0...",acer_campestre
3,"[22.0, 34.0, 54.0, 48.0, 46.0, 40.0, 24.0, 8.0...",acer_campestre
4,"[3.0, 15.0, 27.0, 45.0, 40.0, 40.0, 34.0, 18.0...",acer_campestre
...,...,...
13516,"[39.0, 64.0, 69.0, 89.0, 57.0, 52.0, 17.0, 2.0...",quercus_robur
13517,"[28.0, 68.0, 48.0, 69.0, 68.0, 54.0, 14.0, 3.0...",quercus_robur
13518,"[22.0, 53.0, 69.0, 75.0, 50.0, 42.0, 17.0, 11....",quercus_robur
13519,"[34.0, 84.0, 62.0, 59.0, 58.0, 51.0, 15.0, 5.0...",quercus_robur


In [113]:
test = np.array(lab['int_hist_values'][2])
print(len(test))

525


# Nearest Neighbors - Training

In [114]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [176]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Create a nearest neighbors object with n_neighbors set to 25
nbrs = NearestNeighbors(n_neighbors=15, algorithm='brute', metric='cosine').fit(lab_int)
nbrs

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=15)

In [178]:
input_hist = "[19.  53.  57.  45.  38.  31.  12.   6.   2.   0.   0.   0.   1.   1.  0.   0.   0.   2.   0.   1.   0.  57. 100.  49.  29.   9.   5.   1.  1.   2.   5.   3.   1.   2.   3.   0.   1.   0.   0.   0.   0.   0. 72.  88.  57.  15.   4.   3.   3.   0.   0.   3.   0.   4.   4.   2.  2.   2.   4.   2.   1.   2.   0. 101.  93.  28.   4.   3.   6.   7.  3.   5.   3.   3.   4.   3.   1.   1.   0.   2.   1.   0.   0.   0. 86. 114.  10.   1.   1.   2.   4.   6.   7.   2.   3.   5.   7.   2. 10.   5.   2.   1.   0.   0.   0.  92.  94.  11.   0.   2.   3.   2.  4.   3.   5.   3.   3.   6.   5.   6.  12.   8.   3.   4.   2.   0. 95.  82.  13.   0.   1.   2.   1.   3.  10.   3.   8.   8.   5.   4. 11.  10.   4.   4.   3.   1.   0.  99.  80.   0.   2.   0.   0.   0.  1.   1.   1.   4.   7.  21.  14.  10.  18.   7.   3.   0.   0.   0. 88.  82.   0.   0.   2.   0.   0.   0.   7.   7.   5.   1.   6.  10. 24.  23.   8.   3.   1.   1.   0.  75.  84.   2.   0.   1.   0.   1.  1.   0.   5.   4.   8.   7.   5.  15.  22.  11.   7.  11.   9.   0. 40.  89.   7.   0.   2.   0.   1.   3.   3.   6.  12.  15.   9.   6. 28.  22.  11.   5.   5.   3.   1.  14. 105.   7.   1.   1.   0.   1.  6.  14.  13.   8.  12.  11.  11.  12.  15.  15.  15.   3.   2.   2. 14.  72.  12.   2.   1.   5.   2.   4.  20.  19.  11.  12.  14.   7. 13.   8.   9.   6.  24.  12.   1.   1.  50.  24.  12.   3.  11.  17. 10.  11.  15.  34.  28.   9.   7.   9.   6.   4.   5.   5.   3.   4.  0.   6.  22.  31.  13.  24.  15.  21.  24.  27.  28.  18.  23.   8.  2.   2.   3.   1.   0.   0.   0.   0.   0.   0.   0.   0.  95.  27. 26.  36.  14.  22.   9.   5.  11.   2.   8.   9.   3.   1.   0.   0.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 249.   5.  8.   1.   0.   2.   1.   2.   0.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 268.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0. 268.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 268.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0. 268.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 268.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0. 268.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 268.  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  0.   0.   0.   0.   0.   0. 268.]"
input_hist = input_hist.replace(".", ".0,")
input_hist = ast.literal_eval(input_hist)
input_hist = np.array([input_hist])
query = input_hist.reshape(1, -1)

In [179]:
# Perform the nearest neighbors search
distances, indices = nbrs.kneighbors(query)

In [186]:
import matplotlib.pyplot as plt
from PIL import Image

# Show the top n matches to the user
for i in range(len(indices[0])):
    index = indices[0][i]
    distance = distances[0][i]

    print(f"Match {i} Species: {lab['plant'][index]} Distance: {distance}")

Match 0 Species: acer_campestre Distance: 1.1102230246251565e-16
Match 1 Species: liriodendron_tulipifera Distance: 0.04503782024879788
Match 2 Species: acer_campestre Distance: 0.047309823006039586
Match 3 Species: cercis_canadensis Distance: 0.04851513996352297
Match 4 Species: cercis_canadensis Distance: 0.050423418619843785
Match 5 Species: acer_pensylvanicum Distance: 0.06353159098330186
Match 6 Species: betula_jacqemontii Distance: 0.07030592765242549
Match 7 Species: cercis_canadensis Distance: 0.07261151026631041
Match 8 Species: acer_campestre Distance: 0.07380442161514911
Match 9 Species: acer_saccharum Distance: 0.08063830229877533
Match 10 Species: acer_ginnala Distance: 0.08178600993039764
Match 11 Species: acer_campestre Distance: 0.08220900850347568
Match 12 Species: acer_pseudoplatanus Distance: 0.08333872667939213
Match 13 Species: acer_campestre Distance: 0.08495407190880877
Match 14 Species: acer_campestre Distance: 0.0864264546049438
